# Sistema Embarcado de Previsão Ambiental Multi-Horizonte com MLP

**Objetivo:** Desenvolver um modelo Edge ML (MLP) capaz de:
- **Classificar** a tendência atual de temperatura (Estável, Aquecendo, Resfriando)
- **Prever** temperatura futura em 3 horizontes: 5, 10 e 15 minutos

**Hardware:** BitDogLab (RP2040) com sensores AHT20 e BMP280

---


In [1]:
pip install pandas numpy tensorflow scikit-learn matplotlib seaborn

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(42)
tf.random.set_seed(42)


# 1. Carregar e Explorar os Dados

---


In [5]:
# Carregar o dataset completo
df_raw = pd.read_csv('/content/temp.csv', parse_dates=['Timestamp'])
selected_columns = ['Timestamp', 'Temp_AHT20_C', 'Umid_AHT20_pct', 'Temp_BMP280_C', 'Press_BMP280_hPa']# Definir as colunas que serão utilizadas (Fusão de Sensores: AHT20 + BMP280)
df = df_raw[selected_columns].copy()# Definir as colunas que serão utilizadas (Fusão de Sensores: AHT20 + BMP280) para nossa rede


# Exibir informações do dataset
print(f"Total de amostras: {len(df)}")
print(f"Período: {df['Timestamp'].min()} até {df['Timestamp'].max()}")
print(f"\nColunas selecionadas:\n{df.columns.tolist()}")
print(f"\nPrimeiras linhas:")
print(df.head())

Total de amostras: 82430
Período: 2025-08-31 22:31:02 até 2025-09-30 16:52:09

Colunas selecionadas:
['Timestamp', 'Temp_AHT20_C', 'Umid_AHT20_pct', 'Temp_BMP280_C', 'Press_BMP280_hPa']

Primeiras linhas:
            Timestamp  Temp_AHT20_C  Umid_AHT20_pct  Temp_BMP280_C  \
0 2025-08-31 22:31:02         19.90           73.37          21.28   
1 2025-08-31 22:31:33         19.97           75.57          21.48   
2 2025-08-31 22:32:05         19.75           74.31          21.25   
3 2025-08-31 22:32:36         19.55           74.77          21.07   
4 2025-08-31 22:33:07         19.60           75.09          21.12   

   Press_BMP280_hPa  
0            921.24  
1            921.12  
2            921.20  
3            921.15  
4            921.25  


In [6]:
print(f"\nEstatísticas descritivas:")
print(df[['Temp_AHT20_C', 'Umid_AHT20_pct', 'Temp_BMP280_C', 'Press_BMP280_hPa']].describe())


Estatísticas descritivas:
       Temp_AHT20_C  Umid_AHT20_pct  Temp_BMP280_C  Press_BMP280_hPa
count  82430.000000    82430.000000   82430.000000      82430.000000
mean      20.328970       65.946394      21.803339        918.146819
std        3.452443       13.343084       3.350503          2.147400
min       13.440000       35.440000      15.120000        912.100000
25%       17.610000       54.520000      19.160000        916.600000
50%       19.750000       68.950000      21.240000        918.300000
75%       22.990000       77.310000      24.400000        919.790000
max       29.390000       92.130000      30.590000        922.990000


In [7]:
print(df.isnull().sum())#Verificar se existem valores nulos
# Remover linhas com valores nulos (se existirem)
df_clean = df.dropna()
print(f"\nDados após limpeza: {len(df_clean)} amostras ({len(df) - len(df_clean)} removidas)")

Timestamp           0
Temp_AHT20_C        0
Umid_AHT20_pct      0
Temp_BMP280_C       0
Press_BMP280_hPa    0
dtype: int64

Dados após limpeza: 82430 amostras (0 removidas)


- Vemos que não teve dados nulos.